Last time, we had fun diving into the world of penguins, creating some cool visualizations with the Palmer Penguins dataset. Penguins are adorable little waddlers, but they're facing serious threats from climate change.

That's why it's so important to understand climate change and how it's evolving—and data visualization with Python is an awesome way to bring this understanding to life.

In this tutorial, we're stepping it up! We'll use NOAA's (National Oceanic and Atmospheric Administration) climate dataset. And we'll focus on making interactive maps and plots that uncover temperature patterns and changes over time. Let's get started!

## Preparing our Environment

First, let's set up our Python environment with the necessary libraries. We'll be using:

* ```pandas``` and ```numpy``` for data manipulation
* ```plotly``` for interactive visualizations
* ```sqlite3``` for database operations
* ```scikit-learn``` for statistical analysis

In [5]:
import plotly.io as pio
pio.renderers.default="iframe"
import pandas as pd
import numpy as np
import sqlite3
from plotly import express as px
from sklearn.linear_model import LinearRegression
import calendar
import inspect

## Creating our Database

Our first step is to create a SQLite database to store the NOAA climate data. To accelerate the process, let's download the files to our working directory now.

In [8]:
import os
# create folder named "datafiles" if it does not exist
if not os.path.exists("datafiles"): 
    os.mkdir("datafiles")

# download the files
import urllib.request
intervals = [f"{i}-{i+9}" for i in range(1901, 2020, 10)]
for interval in intervals:
    url = f"https://raw.githubusercontent.com/PIC16B-ucla/25W/main/datasets/noaa-ghcn/decades/{interval}.csv"
    urllib.request.urlretrieve(url, f"datafiles/{interval}.csv")

The data is organized by decades and includes temperature readings from weather stations worldwide.

Let's first create a database in current directory called ```climate.db```.

In [10]:
conn = sqlite3.connect("climate.db")

We'll use a helper function ```prepare_df()``` to transform our data from wide format (where months are columns) to long format (where each month gets its own row). This makes the data easier to analyze and visualize.

We'll also do a simple cleaning of the data by removing any rows with temperature measures of ```NaN```.

In [12]:
def prepare_df(df):
    """
    prepares a piece of wide format dataframe into a long format data frame
    """
    # melt to the long format table
    df = df.melt(
        id_vars = ["ID", "Year"],
        value_vars = [f"VALUE{i}" for i in range(1, 13)],
        var_name = "Month",
        value_name = "Temp"
    )

    # cleaning month and temp
    df["Month"] = df["Month"].str[5:].astype(int)
    df["Temp"]  = df["Temp"] / 100
    df = df[~np.isnan(df["Temp"])]  # Remove rows with missing temperature values

    return df

Now we'll load our temperature data into the database. We'll iterate through each decade file, creating or appending the data to the temperatures table.

In [14]:
intervals = [f"{i}-{i+9}" for i in range(1901, 2020, 10)]
for i, interval in enumerate(intervals):
    filepath = f"datafiles/{interval}.csv"
    df = pd.read_csv(filepath)
    df = prepare_df(df)
    df.to_sql("temperatures", conn, if_exists = "replace" if i == 0 else "append", index = False)

We also need the station metadata. It provides geographic coordinates for each station, station names, and other identifiers.

In [16]:
stations_url = "https://raw.githubusercontent.com/PIC16B-ucla/25W/refs/heads/main/datasets/noaa-ghcn/station-metadata.csv"
stations = pd.read_csv(stations_url)
stations.to_sql("stations", conn, if_exists = "replace", index=False)

27585

Finally, we'll load the country information as well. The country codes table will allow us to map station IDs to countries. We'll relate them to temperature readings later in the next section!

In [18]:
countries_url = "https://raw.githubusercontent.com/mysociety/gaze/master/data/fips-10-4-to-iso-country-codes.csv"
countries = pd.read_csv(countries_url)
countries.to_sql("countries", conn, if_exists = "replace", index=False)

279

Let's verify the content of our database -- there should be three tables: ```temperatures```, ```stations```, and ```countries```. 

In [20]:
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
print(cursor.fetchall())

[('temperatures',), ('stations',), ('countries',)]


Don't forget to close the database connection as we are now finished constructing it.

In [22]:
conn.close()

## Writing a Query Function

Now that the databse is ready, we'll need to write query functions to retrieve and organize the data needed for our visualizations.

Let's first write a function called ```query_climate_database()```, which retrieves temperature data with comprehensive contextual information.

In [25]:
from climate_database import query_climate_database

print(inspect.getsource(query_climate_database))

def query_climate_database(db_file, country, year_begin, year_end, month):
    """
    Parameters:
    -----------
    db_file (str): File name for the database
    country (str): Name of the country
    year_begin (int): Earliest year for the date range (inclusive)
    year_end (int): Lastest year for the date range (inclusive)
    month (int): Month of the year
        
    Returns:
    --------
    pandas.DataFrame
        DataFrame containing temperature readings with columns:
        NAME, LATITUDE, LONGITUDE, Country, Year, Month, Temp
    """
    
    # Create connection to database
    conn = sqlite3.connect(db_file)
    
    # Construct the SQL query using f-strings
    query = f"""
        SELECT 
            s.NAME,
            s.LATITUDE,
            s.LONGITUDE,
            c.Name as Country,
            t.Year,
            t.Month,
            t.Temp
        FROM temperatures t
        JOIN stations s ON t.ID = s.ID
        JOIN countries c ON substr(t.ID, 1, 2) = c."FIPS

This function joins three different tables and returns temperature readings for the specified country, within the specified date range, and for the specified month of the year.

Let's inspect some data using the function. For example, if we want to know the January temperatures in India from 1980 to 2020:

In [27]:
query_climate_database(db_file = "climate.db",
                       country = "India", 
                       year_begin = 1980, 
                       year_end = 2020,
                       month = 1)

,NAME,LATITUDE,LONGITUDE,Country,Year,Month,Temp
0,AGARTALA,23.883,91.250,India,1980,1,18.21
1,AGARTALA,23.883,91.250,India,1981,1,18.25
2,AGARTALA,23.883,91.250,India,1982,1,19.31
3,AGARTALA,23.883,91.250,India,1985,1,19.25
4,AGARTALA,23.883,91.250,India,1988,1,19.54
...,...,...,...,...,...,...,...
3147,VISHAKHAPATNAM,17.717,83.233,India,2016,1,25.09
3148,VISHAKHAPATNAM,17.717,83.233,India,2017,1,23.90
3149,VISHAKHAPATNAM,17.717,83.233,India,2018,1,22.65
3150,VISHAKHAPATNAM,17.717,83.233,India,2019,1,22.20


## Making a Geographic Scatterplot for Yearly Temperature Increases

Here comes the exciting part: let's create our first visualization. For example, if we are interested in this question:

> **How does the average yearly change in temperature vary within a given country?**

We can create an interactive geographic scatterplot, constructed using Plotly Express.

There will be a point for each station, such that the color of the point reflects an estimate of the yearly change in temperature during the specified month and time period at that station. 

In [32]:
from main import temperature_coefficient_plot

print(inspect.getsource(temperature_coefficient_plot))

def temperature_coefficient_plot(db_file, country, year_begin, year_end, month, min_obs, **kwargs):
    """
    Parameters:
    -----------
    db_file (str): File name for the database
    country (str): Name of the country
    year_begin (int): Earliest year for the date range (inclusive)
    year_end (int): Latest year for the date range (inclusive)
    month (int): Month of the year
    min_obs (int): Minimum number of years of data required for a station to be included
    **kwargs: Additional keyword arguments to pass to px.scatter_mapbox()
    
    Returns:
    --------
    plotly.graph_objects.Figure
    """
    
    # Query the database
    df = query_climate_database(db_file, country, year_begin, year_end, month)
    
    # Filter for stations with minimum required observations
    df = df.groupby(['NAME', 'LATITUDE', 'LONGITUDE']).filter(lambda x: len(x) >= min_obs)
    
    # Calculate temperature coefficients using linear regression
    def calc_coefficient(group):
       

Now, we can create a plot of the estimated yearly increases in temperature during the month of January, for the interval 1980-2020, in India, for example:

In [34]:
color_map = px.colors.diverging.RdGy_r

fig = temperature_coefficient_plot("climate.db", "India", 1980, 2020, 1, 
                                   min_obs = 10,
                                   zoom = 2,
                                   mapbox_style="carto-positron",
                                   color_continuous_scale=color_map)

fig.show()

Since the visualization is wrapped in functions, we can create visualizations for different parts of the data by calling these functions with different arguments. For example, we can also create a plot of the estimate yearly increases in temperature in September, for the interval 1990-2000, in China:

In [36]:
color_map = px.colors.diverging.Tealrose

fig = temperature_coefficient_plot("climate.db", "China", 1990, 2000, 9, 
                                   min_obs = 10,
                                   zoom = 2,
                                   mapbox_style="carto-positron",
                                   color_continuous_scale=color_map)

fig.show()

## Make Two More Interesting Figures

We can also create two more complex and interesting interactive data visualizations using the same data set!

For example, we might be interested in the following question:

> **How do seasonal temperature patterns vary between northern and southern regions of a country?**

Similar to the previous sections, we can write a query function to facilitate the retrieval of relevant data for analyzing this question.

The function ```query_regional_patterns()``` will divide the country into North, Central, and South region and aggregate data by month for seasonal analysis.

In [41]:
from climate_database import query_regional_patterns

print(inspect.getsource(query_regional_patterns))

def query_regional_patterns(db_file, country, year_begin, year_end):
    """
    Parameters:
    -----------
    db_file (str): File name for the database
    country (str): Name of the country
    year_begin (int): Earliest year for the date range (inclusive)
    year_end (int): Latest year for the date range (inclusive)
    
    Returns:
    --------
    pandas.DataFrame with columns:
        Year: year of measurement
        Month: month of measurement
        region: geographical region (North, Central, South)
        temperature: average temperature for that region/month/year
    """
    conn = sqlite3.connect(db_file)
    
    query = f"""
        WITH station_regions AS (
            SELECT 
                s.ID,
                CASE 
                    WHEN s.LATITUDE >= LAT_MAX - (LAT_MAX - LAT_MIN)/3 THEN 'North'  -- North region if latitude is at the upper third of the range
                    WHEN s.LATITUDE >= LAT_MAX - 2*(LAT_MAX - LAT_MIN)/3 THEN 'Central'  -- Central 

We can use a faceted line plot to show monthly temperature patterns for different regions of the country. The line plots can show continuous trends over time, and faceting will separate data by region for easy comparison.

In [43]:
from main import plot_regional_patterns

print(inspect.getsource(plot_regional_patterns))

def plot_regional_patterns(db_file, country, year_begin, year_end):
    """
    Parameters:
    -----------
    db_file (str): File name for the database
    country (str): Name of the country
    year_begin (int): Earliest year for the date range (inclusive)
    year_end (int): Latest year for the date range (inclusive)
    
    Returns:
    --------
    plotly.graph_objects.Figure
    """
    # Query the data
    df = query_regional_patterns(db_file, country, year_begin, year_end)
    
    # Add month names
    df['MonthName'] = df['Month'].apply(lambda x: calendar.month_abbr[x])
    
    # Calculate regional averages by month
    monthly_avgs = df.groupby(['region', 'Month', 'MonthName'])['Temp'].mean().reset_index()
    
    # Create faceted line plot
    fig = px.line(
        monthly_avgs,
        x='MonthName',
        y='Temp',
        facet_col='region',
        title=f'Monthly Temperature Patterns by Region in {country} ({year_begin}-{year_end})',
        labels={
           

Now, we can visualize the data, for example, the regional temperature patterns in India from 1960 to 2020:

In [45]:
fig = plot_regional_patterns("climate.db",
                           country="India",
                           year_begin=1960,
                           year_end=2020)
fig.show()

Not bad! Now, we can move on to explore another question:

> **Are winters warming faster than summers in different regions of a country?**

Again, we can first write a query function ```query_regional_warming()``` to identify summer (Jun-Aug) and winter (Dec-Feb) months and prepare data for trend analysis.

In [49]:
from climate_database import query_regional_warming

print(inspect.getsource(query_regional_warming))

def query_regional_warming(db_file, country, year_begin, year_end):
    """
    Parameters:
    -----------
    db_file (str): File name for the database
    country (str): Name of the country
    year_begin (int): Earliest year for the date range (inclusive)
    year_end (int): Latest year for the date range (inclusive)
    
    Returns:
    --------
    pandas.DataFrame with regional summer/winter temperatures over time
    """
    conn = sqlite3.connect(db_file)
    
    query = f"""
        WITH station_regions AS (
            SELECT 
                s.ID,
                CASE 
                    WHEN s.LATITUDE >= LAT_MAX - (LAT_MAX - LAT_MIN)/3 THEN 'North'
                    WHEN s.LATITUDE >= LAT_MAX - 2*(LAT_MAX - LAT_MIN)/3 THEN 'Central'
                    ELSE 'South'
                END as region
            FROM stations s
            JOIN countries c ON substr(s.ID, 1, 2) = c."FIPS 10-4"
            CROSS JOIN (
                SELECT 
                    MAX(LATITUD

In [50]:
from main import plot_regional_warming

print(inspect.getsource(plot_regional_warming))

def plot_regional_warming(db_file, country, year_begin, year_end):
    """
    Parameters:
    -----------
    db_file (str): File name for the database
    country (str): Name of the country
    year_begin (int): Earliest year for the date range (inclusive)
    year_end (int): Latest year for the date range (inclusive)
    
    Returns:
    --------
    plotly.graph_objects.Figure
    """
    # Query data
    df = query_regional_warming(db_file, country, year_begin, year_end)
    
    # Create faceted scatter plot with trend lines
    fig = px.scatter(
        df,
        x='Year',
        y='Temp',
        color='season',
        facet_col='region',
        trendline="ols",
        title=f'Summer vs Winter Warming Rates by Region in {country} ({year_begin}-{year_end})',
        labels={
            'Year': 'Year',
            'Temp': 'Temperature (°C)',
            'season': 'Season',
            'region': 'Region'
        },
        height=500
    )
    
    return fig



And here, we can use a scatter plot with regression lines to show temperature trends by region, comparing summer and winter warming rates.

In [52]:
fig = plot_regional_warming("climate.db",
                           country="India",
                           year_begin=1960,
                           year_end=2020)
fig.show()

Great! We've created another interactive plot.

Just to recap what we've accomplished in this tutorial: Through a combination of SQL querying, data processing, and Plotly's visualization capabilities, we have learned how to create interactive maps and plots that reveal temperature patterns across different regions and timeframes.

Using similar methods, we can also explore other parts of the data (or even introduce a brand new dataset!) to investigate other aspects of our changing climate.